In [1]:
try:                 
    get_ipython().magic(u'load_ext autoreload')    
    get_ipython().magic(u'autoreload 2')           
    get_ipython().magic(u'matplotlib qt')          
except:              
    pass             
import logging       
import matplotlib.pyplot as plt                    
import numpy as np   
                     
logging.basicConfig(format= 
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",  
                    level=logging.DEBUG)           
                     
import caiman as cm  
from caiman.source_extraction import cnmf                 
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect        
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2           
                     
try:                 
    cv2.setNumThreads(0)
except:              
    pass             
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()                                                                                                            
                     
import glob          
import re            
import os

/home/qiushou/.conda/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qiushou/.conda/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qiushou/.conda/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qiushou/.conda/envs/caiman/lib/python3.7/site-

Loading BokehJS ...

In [34]:
animal_id = '191172' 
resultDir = '/home/qiushou/Documents/QS_data/miniscope/miniscope_result'
tsFileList = glob.glob(os.path.join('/run/user/1000/gvfs/smb-share:server=10.10.46.135,share=data_archive/qiushou/miniscope/20191110'
                                    ,animal_id     
                                    ,"H*/timestamp.dat"))
def sort_key(s):     
    if s:            
        try:         
            date = re.findall('\d{8}', s)[0]
        except:      
            date = -1            
        try:         
            H = re.findall('H(\d+)',s)[0]
        except:      
            H = -1            
        try:         
            M = re.findall('M(\d+)',s)[0]
        except:      
            M = -1            
        try:         
            S = re.findall('S(\d+)',s)[0]
        except:      
            S = -1            
        try:         
            ms = re.findall('msCam(\d+)',s)[0]
        except:      
            ms = -1  
        return [int(date),int(H),int(M),int(S),int(ms)]
tsFileList.sort(key=sort_key)
print(tsFileList)

['/run/user/1000/gvfs/smb-share:server=10.10.46.135,share=data_archive/qiushou/miniscope/20191110/191172/H15_M27_S28/timestamp.dat', '/run/user/1000/gvfs/smb-share:server=10.10.46.135,share=data_archive/qiushou/miniscope/20191110/191172/H15_M41_S35/timestamp.dat']


In [35]:
import cv2          
import pickle       
                    
newpath_parent=os.path.join(resultDir,'Results'+'_'+animal_id)
newpath = os.path.join(newpath_parent,"20191111_093330_20191110")

#%% concatenate timestamps of tsFileList
ms_ts_name = os.path.join(newpath,'ms_ts.pkl')
print(ms_ts_name)  
temporal_downsampling=3                   
import pandas as pd
import pickle   

/home/qiushou/Documents/QS_data/miniscope/miniscope_result/Results_191172/20191111_093330_20191110/ms_ts.pkl


In [36]:
if not os.path.exists(ms_ts_name):
    ts_session=[]  
    for tsFile in tsFileList:
        datatemp=pd.read_csv(tsFile,sep = "\t", header = 0)
        ts_session.append(datatemp['sysClock'].values) 
#         print(len((datatemp['sysClock'].values)))
    ttemp=np.hstack(ts_session)[::temporal_downsampling]
    # remporally downsample for each video
    # [i[::3] for i in ts_session][0]
    session_indend=(np.where(np.diff(ttemp)<0)[0]).tolist()    
    session_indend.append(-1)
    ts_session_ds=[]
    i0=0            
    session_indstart=[]
    for i in range(len(ts_session)):
#         print(i)
        session_indstart.append(i0)
        ts_session_ds.append(ttemp[i0:session_indend[i]])
        i0=session_indend[i]+1
    ms_ts=np.array(ts_session_ds)    
    with open(ms_ts_name,'wb') as output:
        pickle.dump(ms_ts,output,pickle.HIGHEST_PROTOCOL)
else:               
    with open(ms_ts_name, "rb") as f:
        ms_ts= pickle.load(f)
print(f'concatenated timestamp of miniscope_video is located at {ms_ts_name}')

concatenated timestamp of miniscope_video is located at /home/qiushou/Documents/QS_data/miniscope/miniscope_result/Results_191172/20191111_093330_20191110/ms_ts.pkl


In [37]:
print(len(ms_ts[0])+len(ms_ts[1]))

17039


In [50]:
if not os.path.exists(ms_ts_name):
    ts_session=[]
    for tsFile in tsFileList:
        datatemp=pd.read_csv(tsFile,sep = "\t", header = 0)
        ts_session.append(datatemp['sysClock'].values)    
    ttemp=np.hstack(ts_session)[::temporal_downsampling]
    # remporally downsample for each video
    # [i[::3] for i in ts_session][0]
    session_indend=(np.where(np.diff(ttemp)<0)[0]).tolist()
#    session_indend.append(-1)
    ts_session_ds=[]
    i0=0
    session_indstart=[]
    if len(session_indend)>0:
        for i in range(len(session_indend)):
            session_indstart.append(i0)
            ts_session_ds.append(ttemp[i0:(session_indend[i]+1)])
            i0=session_indend[i]+1
        ts_session_ds.append(ttemp[(session_indend[-1]+1):])
    else:
        ts_session_ds.append(ttemp[i0:])
    
    
    ms_ts=np.array(ts_session_ds)    
    with open(ms_ts_name,'wb') as output:
        pickle.dump(ms_ts,output,pickle.HIGHEST_PROTOCOL)
else:
    with open(ms_ts_name, "rb") as f:
        ms_ts= pickle.load(f)
print(f'concatenated timestamp of miniscope_video is located at {ms_ts_name}')

concatenated timestamp of miniscope_video is located at /home/qiushou/Documents/QS_data/miniscope/miniscope_result/Results_191172/20191111_093330_20191110/ms_ts.pkl


In [51]:
sum([len(i) for i in ms_ts]) 

17041

In [54]:
import cv2
cap = cv2.VideoCapture(os.path.join(newpath_parent,"20191111_093330_20191110/msCam_concat.avi"))
int(cap.get(7))

17041

In [57]:
sum([len(i) for i in ms_ts]) == int(cap.get(7))

True